![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Healthcare/6.Clinical_Context_Spell_Checker.ipynb)

<H1> 6. Context Spell Checker - Medical </H1>


In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [ ]:
from johnsnowlabs import nlp, medical, visual

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

👌 Detected license file /content/4.2.4.spark_nlp_for_healthcare.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==4.2.4, 💊Spark-Healthcare==4.2.4, running on ⚡ PySpark==3.1.2


In [ ]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.RecursiveTokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")\
    .setPrefixes(["\"", "(", "[", "\n"])\
    .setSuffixes([".", ",", "?", ")","!", "'s"])

spellModel = nlp.ContextSpellCheckerModel\
    .pretrained('spellcheck_clinical', 'en', 'clinical/models')\
    .setInputCols("token")\
    .setOutputCol("checked")

spellcheck_clinical download started this may take some time.
Approximate size to download 134.7 MB
[OK!]


In [ ]:
pipeline = nlp.Pipeline(
    stages = [
    documentAssembler,
    tokenizer,
    spellModel
    ])

empty_ds = spark.createDataFrame([[""]]).toDF("text")

lp = nlp.LightPipeline(pipeline.fit(empty_ds))

Ok!, at this point we have our spell checking pipeline as expected. Let's see what we can do with it, see these errors,

_She was **treathed** with a five day course of **amoxicilin** for a **resperatory** **truct** infection._

_With pain well controlled on **orall** **meditation**, she was discharged to **reihabilitation** **facilitay**._


_Her **adominal** examination is soft, nontender, and **nonintended**_

_The patient was seen by the **entocrinology** service and she was discharged on 40 units of **unsilin** glargine at night_
      
_No __cute__ distress_

Check that some of the errors are valid English words, only by considering the context the right choice can be made.

In [ ]:
example = ["She was treathed with a five day course of amoxicilin for a resperatory truct infection . ",
           "With pain well controlled on orall meditation, she was discharged to reihabilitation facilitay.",
           "Her adominal examination is soft, nontender, and nonintended.",
           "The patient was seen by the entocrinology service and she was discharged on 40 units of unsilin glargine at night",
           "No cute distress",
          ]

for pairs in lp.annotate(example):
    print(list(zip(pairs['token'],pairs['checked'])))

[('She', 'She'), ('was', 'was'), ('treathed', 'treated'), ('with', 'with'), ('a', 'a'), ('five', 'five'), ('day', 'day'), ('course', 'course'), ('of', 'of'), ('amoxicilin', 'amoxicillin'), ('for', 'for'), ('a', 'a'), ('resperatory', 'respiratory'), ('truct', 'tract'), ('infection', 'infection'), ('.', '.')]
[('With', 'With'), ('pain', 'pain'), ('well', 'well'), ('controlled', 'controlled'), ('on', 'on'), ('orall', 'oral'), ('meditation', 'medication'), (',', ','), ('she', 'she'), ('was', 'was'), ('discharged', 'discharged'), ('to', 'to'), ('reihabilitation', 'rehabilitation'), ('facilitay', 'facility'), ('.', '.')]
[('Her', 'Her'), ('adominal', 'abdominal'), ('examination', 'examination'), ('is', 'is'), ('soft', 'soft'), (',', ','), ('nontender', 'nontender'), (',', ','), ('and', 'and'), ('nonintended', 'nondistended'), ('.', '.')]
[('The', 'The'), ('patient', 'patient'), ('was', 'was'), ('seen', 'seen'), ('by', 'by'), ('the', 'the'), ('entocrinology', 'endocrinology'), ('service', 'se

In [ ]:
print("Corrected tokens:\n")

pair_list = [list(zip(pairs['token'],pairs['checked'])) for pairs in lp.annotate(example)]
corrected_list = [i for pair in pair_list for i in pair if i[0] != i[1]]
corrected_list

Corrected tokens:



[('treathed', 'treated'),
 ('amoxicilin', 'amoxicillin'),
 ('resperatory', 'respiratory'),
 ('truct', 'tract'),
 ('orall', 'oral'),
 ('meditation', 'medication'),
 ('reihabilitation', 'rehabilitation'),
 ('facilitay', 'facility'),
 ('adominal', 'abdominal'),
 ('nonintended', 'nondistended'),
 ('entocrinology', 'endocrinology'),
 ('unsilin', 'insulin'),
 ('cute', 'acute')]